In [1]:
# Import libraries
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout

In [2]:
df_for_training = pd.read_csv("/content/drive/MyDrive/Datasets/Project/df_for_training.csv")

In [3]:
df_for_training.shape

(24548, 57)

In [4]:
df_for_training.columns[:7]

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Sentiments'], dtype='object')

In [5]:
# Function to create sequences
def singleStepSampler(df, window):
	xRes = []
	yRes = []
	for i in range(0, len(df) - window):
		res = []
		for j in range(0, window):
			r = []
			for col in df.columns:
				r.append(df[col][i + j])
			res.append(r)
		xRes.append(res)
		yRes.append(df[['Open', 'Close', 'High', 'Low', 'Adj Close', 'Volume', 'Sentiments']].iloc[i + window].values)
	return np.array(xRes), np.array(yRes)

In [ ]:
# Dataset splitting
SPLIT = 1.0
(xVal, yVal) = singleStepSampler(df_for_training, 20)
X_train = xVal[:int(SPLIT * len(xVal))]
y_train = yVal[:int(SPLIT * len(yVal))]
X_test = xVal[int(SPLIT * len(xVal)):]
y_test = yVal[int(SPLIT * len(yVal)):]
print(xVal.shape)
print(yVal.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [16]:
# Building the model

multivariate_lstm = keras.Sequential()

# First LSTM layer to capture short-term dependencies within the window
multivariate_lstm.add(keras.layers.LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
multivariate_lstm.add(keras.layers.Dropout(0.2))  # Dropout for regularization

# Second LSTM layer to capture potential hierarchical features
multivariate_lstm.add(keras.layers.LSTM(100))  # Experiment with different units here
multivariate_lstm.add(keras.layers.Dropout(0.2))  # Dropout for regularization

# Dense layer with ReLU activation for non-linearity
multivariate_lstm.add(keras.layers.Dense(7, activation='relu'))

# Compile the model with Adam optimizer and Mean Squared Error loss
multivariate_lstm.compile(loss = 'MeanSquaredError', metrics=['MAE'], optimizer='Adam')

# Print model summary
multivariate_lstm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 20, 200)           206400    
                                                                 
 dropout_6 (Dropout)         (None, 20, 200)           0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               120400    
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 7)                 707       
                                                                 
Total params: 327507 (1.25 MB)
Trainable params: 327507 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Fitting the data to the model
history = multivariate_lstm.fit(X_train, y_train, epochs=100)

Epoch 1/100
767/767 [==============================] - 33s 39ms/step - loss: 0.0082 - MAE: 0.0588
Epoch 2/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0049 - MAE: 0.0433
Epoch 3/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0042 - MAE: 0.0386
Epoch 4/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0039 - MAE: 0.0359
Epoch 5/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0038 - MAE: 0.0349
Epoch 6/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0036 - MAE: 0.0335
Epoch 7/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0036 - MAE: 0.0331
Epoch 8/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0035 - MAE: 0.0326
Epoch 9/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0034 - MAE: 0.0322
Epoch 10/100
767/767 [==============================] - 30s 39ms/step - loss: 0.0034 - MAE: 0.0319
Epoch 11/100
767/76

In [12]:
# Save the model to a specific path
model_path = '/content/drive/MyDrive/Final Year Project/multivariate_lstm_model4.h5'
multivariate_lstm.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
